In [1]:
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM, Dense, Flatten, BatchNormalization, Dropout, GRU
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import module
import gc
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
np.random.randint(10)

2

In [3]:
countriesOfInterest = {"HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ'}
countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"]
#countriesOfInterest = list(countriesOfInterest & {"ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"})

enc = OneHotEncoder()
enc.fit(np.array(countriesOfInterest).reshape(-1, 1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [4]:
seconds = 10
samplerate = 10
train_x, train_labels, val_x, val_labels, class_weights = module.getSamples(1, 1, 1, 1, seconds, samplerate, countriesOfInterest,
               enc, verbose = 0)
print("train", np.sum(train_labels, axis = 0))

train [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Fit model

In [5]:
input_shape=(train_x.shape[1], train_x.shape[2])
input_shape

(100, 25)

In [ ]:
train_n = 400
sample_n = 33000
val_n  = 100
valsample_n = 3000
iterations = 10
epochs = 10
learn_rate = 0.001
batch_size = 4096 * 2
for D in [8, 16, 32, 64]:
    for LSTM2 in [8, 16, 32, 64]:
        for LSTM1 in [8, 16, 32, 64]:
            model = keras.Sequential()
            model.add(BatchNormalization(input_shape=input_shape))

            model.add(LSTM(LSTM1, 
                           input_shape=(train_x.shape[1], train_x.shape[2]), 
                           return_sequences = True
                          ))
            model.add(BatchNormalization())
            model.add(LSTM(LSTM2, 
                           return_sequences = False,
                           go_backwards = True
                          ))
            model.add(BatchNormalization())
            model.add(Dense(D, activation= "relu"))
            model.add(BatchNormalization())
            model.add(Dense(len(enc.categories_[0]), activation= "softmax"))
            adam = keras.optimizers.Adam(lr=0.001, amsgrad = True)
            model.compile(loss = "categorical_crossentropy", optimizer= adam, metrics=["acc"])
            del train_x, train_labels, val_x, val_labels, class_weights
            gc.collect()
            print(model.summary())
            desc = str(LSTM1) + "_" + str(LSTM2) + "_" + str(D)
            log_dir = os.path.join(
                "logs",
                "10sec_10sample",
                desc
            )
            model_dir = os.path.join(
                "pickle",
                desc
            )


            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            model = module.train(iterations, learn_rate, train_n, sample_n, val_n, valsample_n, seconds, samplerate,
                            countriesOfInterest, enc, epochs, tensorboard_callback, model_dir,
                            model, batch_size)
            del model, model_dir, log_dir, desc, adam
            gc.collect()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 100, 25)           100       
_________________________________________________________________
lstm (LSTM)                  (None, 100, 8)            1088      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 8)            32        
_________________________________________________________________
lstm_1 (LSTM)                (None, 8)                 544       
_________________________________________________________________
batch_normalization_2 (Batch (None, 8)                 32        
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
batch_normalization_3 (Batch (None, 8)                 3

In [ ]:
model = keras.Sequential()
model.add(BatchNormalization(input_shape=(train_x.shape[1], train_x.shape[2])))

model.add(LSTM(16, 
               input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = True,
          #     recurrent_dropout = .5,
          #     kernel_regularizer=regularizers.l2(0.01),
              ))
model.add(BatchNormalization())
model.add(LSTM(32, 
             #  input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = False,
  #             recurrent_dropout = .5,
               go_backwards = True
          #     kernel_regularizer=regularizers.l2(0.01),
              ))
model.add(BatchNormalization())
#model.add(Dropout(.5))
model.add(Dense(16, activation= "relu"))
model.add(BatchNormalization())
#model.add(Dropout(.5))
model.add(Dense(len(enc.categories_[0]), activation= "softmax"))
adam = keras.optimizers.Adam(lr=0.001, amsgrad = True)
model.compile(loss = "categorical_crossentropy", optimizer= adam, metrics=["acc"])
del train_x, train_labels, val_x, val_labels, class_weights
gc.collect()
print(model.summary())

In [ ]:
#add recurrent dropout to add noise to duration
desc = "LSTM16_32_D16_Out_1Step_001LR_amsgrad"
log_dir = os.path.join(
    "logs",
    "Overfit",
    desc
)
model_dir = os.path.join(
    "pickle",
    "save"
)

train_n = 400
sample_n = 33000
val_n  = 100
valsample_n = 3000
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
epochs = 1
iterations = 40
learn_rate = 0.001
batch_size = 4096
model = module.train(iterations, learn_rate, train_n, sample_n, val_n, valsample_n, seconds, samplerate,
                countriesOfInterest, enc, epochs, tensorboard_callback, model_dir,
                model, batch_size)